In [3]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', family='AppleGothic')
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False

from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage,dendrogram,fcluster
from sklearn.cluster import KMeans,DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

from IPython.display import display
pd.options.display.max_columns = None

In [5]:
mfile = pd.read_csv('mfile_merge2.csv')
mfile.head()

,회원번호,조합원상태,성별,연령,배송서비스신청여부,모바일알람여부,Gold_member,연령대,구매금액,구,동,연령60이상,방문횟수,연령대_4560대,구매매장,간식,건강,과실,기타,반찬,생활용품,서류,수산,양념/가루,음료,잡곡,주곡,주류,채소,축산물,주말,평일,1회당평균구매물품수
0,1000012800,1,0,58.0,0,0,0,50대,7658.0,수지구,풍덕천동,0,1,1,매장B,0.000000,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1.000000,2.000000
1,1000103744,1,0,34.0,0,0,0,30대,36948.0,처인구,고림동,0,1,0,매장C,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.000000,0.000000,7.000000
2,1000122304,1,0,38.0,0,0,0,30대,142831.0,광주,기타,0,15,0,매장C,0.083333,0.0,0.0,0.0,0.250000,0.000000,0.041667,0.000000,0.083333,0.000000,0.041667,0.0,0.0,0.333333,0.166667,0.466667,0.533333,1.600000
3,1000163136,1,0,73.0,0,0,0,70대이상,138892.0,수지구,동천동,1,4,0,매장A,0.000000,0.0,0.0,0.0,0.153846,0.307692,0.000000,0.307692,0.000000,0.000000,0.000000,0.0,0.0,0.076923,0.153846,0.250000,0.750000,3.250000
4,1000281920,1,0,42.0,1,0,0,40대,632554.0,수지구,풍덕천동,0,28,1,매장C,0.155556,0.0,0.0,0.0,0.111111,0.022222,0.044444,0.022222,0.044444,0.111111,0.000000,0.0,0.0,0.200000,0.288889,0.464286,0.535714,1.607143


In [29]:
dist_A = pd.read_csv('A매장_거리.csv')
dist_A

,위치,거리,구
0,광주-기타,29,광주
1,광주-산성동,24,광주
2,광주-오포읍,20,광주
3,기타-금광동,25,중원구
4,기타-기타,17,성남
...,...,...,...
68,처인구-역북동,24,처인구
69,처인구-원삼면,45,처인구
70,처인구-이동면,31,처인구
71,하남-기타,34,하남


In [31]:
dist_A['주소'] = dist_A.위치
dist_A.drop('위치', axis = 1, inplace = True)

In [7]:
memberA = mfile[mfile['구매매장'] == '매장A']
memberB = mfile[mfile['구매매장'] == '매장B']
memberC = mfile[mfile['구매매장'] == '매장C']
memberD = mfile[mfile['구매매장'] == '매장D']

In [23]:
member = pd.read_csv('member_data.csv', header = 1)

In [26]:
mem_add = member.loc[:, ['회원번호', '주소']]
mem_add

,회원번호,주소
0,1000012800,수지구-풍덕천동
1,1000103744,처인구-고림동
2,1000122304,광주-기타
3,1000163136,수지구-동천동
4,1000281920,수지구-풍덕천동
...,...,...
11550,999682432,수지구-동천동
11551,999721408,수지구-동천동
11552,999738112,수지구-성복동
11553,999754816,수지구-상현동


In [37]:
dist_A = dist_A.merge(mem_add, on = '주소')
dist_A

,거리,구,주소,회원번호
0,29,광주,광주-기타,1000122304
1,29,광주,광주-기타,1120866240
2,29,광주,광주-기타,1211296128
3,29,광주,광주-기타,1296956096
4,29,광주,광주-기타,133593024
...,...,...,...,...
11498,51,화성,화성-기타,959028608
11499,51,화성,화성-기타,963592512
11500,51,화성,화성-기타,963776256
11501,51,화성,화성-기타,965932928


In [40]:
mA = memberA.merge(dist_A.loc[:,['거리','회원번호']], on = '회원번호' , how = 'left')
mA

,회원번호,조합원상태,성별,연령,배송서비스신청여부,모바일알람여부,Gold_member,연령대,구매금액,구,동,연령60이상,방문횟수,연령대_4560대,구매매장,간식,건강,과실,기타,반찬,생활용품,서류,수산,양념/가루,음료,잡곡,주곡,주류,채소,축산물,주말,평일,1회당평균구매물품수,거리
0,1000163136,1,0,73.0,0,0,0,70대이상,138892.0,수지구,동천동,1,4,0,매장A,0.000000,0.000000,0.000000,0.0,0.153846,0.307692,0.000000,0.307692,0.000000,0.000000,0.000000,0.000000,0.0,0.076923,0.153846,0.250000,0.750000,3.250000,3
1,1000491648,1,0,37.0,0,0,0,30대,663152.0,수지구,동천동,0,32,0,매장A,0.176849,0.003215,0.016077,0.0,0.080386,0.064309,0.000000,0.048232,0.112540,0.128617,0.000000,0.000000,0.0,0.176849,0.192926,0.343750,0.656250,1.943750,3
2,1000508352,1,0,35.0,0,0,0,30대,7083.0,기흥구,상하동,0,1,0,매장A,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,1.000000,22
3,1000577024,1,0,47.0,0,0,0,40대,218435.0,기타,기타,0,3,1,매장A,0.000000,0.000000,0.052632,0.0,0.000000,0.105263,0.000000,0.000000,0.052632,0.631579,0.052632,0.000000,0.0,0.000000,0.105263,0.000000,1.000000,6.333333,17
4,1000796032,1,0,40.0,0,0,0,40대,117067.0,수지구,동천동,0,6,1,매장A,0.083333,0.000000,0.000000,0.0,0.416667,0.000000,0.083333,0.000000,0.000000,0.166667,0.166667,0.083333,0.0,0.000000,0.000000,0.333333,0.666667,2.000000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165,999593344,1,0,49.0,0,0,0,40대,3733.0,수지구,동천동,0,1,1,매장A,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,1.000000,3
2166,999615616,1,0,48.0,0,0,0,40대,7083.0,수지구,동천동,0,1,1,매장A,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,1.000000,3
2167,999641600,1,0,41.0,0,0,0,40대,6126.0,수지구,동천동,0,1,1,매장A,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,1.000000,1.000000,3
2168,999682432,0,0,47.0,0,0,0,40대,39246.0,수지구,동천동,0,1,1,매장A,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,1.000000,2.000000,3


In [42]:
dist_B = pd.read_csv('B매장_거리.csv')
dist_B['주소'] = dist_B.위치
dist_B.drop('위치', axis = 1, inplace = True)
dist_B = dist_B.merge(mem_add, on = '주소')
mB = memberB.merge(dist_B.loc[:,['거리','회원번호']], on = '회원번호' , how = 'left')
mB

,회원번호,조합원상태,성별,연령,배송서비스신청여부,모바일알람여부,Gold_member,연령대,구매금액,구,동,연령60이상,방문횟수,연령대_4560대,구매매장,간식,건강,과실,기타,반찬,생활용품,서류,수산,양념/가루,음료,잡곡,주곡,주류,채소,축산물,주말,평일,1회당평균구매물품수,거리
0,1000012800,1,0,58.0,0,0,0,50대,7658.0,수지구,풍덕천동,0,1,1,매장B,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,2.000000,7
1,1001982016,1,0,52.0,0,0,0,50대,22016.0,수지구,상현동,0,2,1,매장B,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.500000,0.500000,0.500000,0.500000,2.000000,4
2,1002106368,1,0,40.0,0,0,0,40대,363546.0,분당구,판교동,0,15,1,매장B,0.000000,0.000000,0.000000,0.0,0.133333,0.000000,0.000000,0.066667,0.133333,0.033333,0.033333,0.000000,0.0,0.466667,0.133333,0.266667,0.733333,2.000000,14
3,100224000,1,0,63.0,0,0,0,60대,934712.0,수지구,상현동,1,43,1,매장B,0.036364,0.000000,0.027273,0.0,0.190909,0.009091,0.027273,0.036364,0.063636,0.000000,0.036364,0.036364,0.0,0.490909,0.045455,0.930233,0.069767,2.558140,4
4,1004699200,1,0,40.0,0,0,0,40대,23930.0,수지구,풍덕천동,0,2,1,매장B,0.000000,0.000000,0.000000,0.0,0.750000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,2.000000,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2958,999574784,1,0,35.0,0,0,0,30대,281035.0,수지구,상현동,0,13,0,매장B,0.333333,0.041667,0.000000,0.0,0.083333,0.041667,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.153846,0.846154,1.846154,4
2959,999582208,1,0,40.0,0,0,0,40대,10338.0,수지구,상현동,0,2,1,매장B,0.500000,0.000000,0.000000,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,1.000000,4
2960,999624896,1,0,54.0,0,0,0,50대,1914.0,기흥구,보정동,0,1,1,매장B,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,1.000000,7
2961,999754816,1,0,44.0,1,0,0,40대,641334.0,수지구,상현동,0,21,1,매장B,0.156863,0.000000,0.000000,0.0,0.098039,0.039216,0.000000,0.078431,0.000000,0.039216,0.000000,0.000000,0.0,0.235294,0.352941,0.190476,0.809524,2.428571,4


In [44]:
dist_C = pd.read_csv('C매장_거리.csv')
dist_C['주소'] = dist_C.위치
dist_C.drop('위치', axis = 1, inplace = True)
dist_C = dist_C.merge(mem_add, on = '주소')
mC = memberC.merge(dist_C.loc[:,['거리','회원번호']], on = '회원번호' , how = 'left')
mC

,회원번호,조합원상태,성별,연령,배송서비스신청여부,모바일알람여부,Gold_member,연령대,구매금액,구,동,연령60이상,방문횟수,연령대_4560대,구매매장,간식,건강,과실,기타,반찬,생활용품,서류,수산,양념/가루,음료,잡곡,주곡,주류,채소,축산물,주말,평일,1회당평균구매물품수,거리
0,1000103744,1,0,34.0,0,0,0,30대,36948.0,처인구,고림동,0,1,0,매장C,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.000000,0.000000,7.000000,19
1,1000122304,1,0,38.0,0,0,0,30대,142831.0,광주,기타,0,15,0,매장C,0.083333,0.0,0.0,0.0,0.250000,0.000000,0.041667,0.000000,0.083333,0.000000,0.041667,0.0,0.0,0.333333,0.166667,0.466667,0.533333,1.600000,26
2,1000281920,1,0,42.0,1,0,0,40대,632554.0,수지구,풍덕천동,0,28,1,매장C,0.155556,0.0,0.0,0.0,0.111111,0.022222,0.044444,0.022222,0.044444,0.111111,0.000000,0.0,0.0,0.200000,0.288889,0.464286,0.535714,1.607143,2
3,1000565888,1,0,52.0,0,0,0,50대,49296.0,수지구,상현동,0,5,1,매장C,0.000000,0.0,0.0,0.0,0.111111,0.333333,0.000000,0.444444,0.000000,0.000000,0.000000,0.0,0.0,0.111111,0.000000,0.200000,0.800000,1.800000,7
4,1001042880,1,0,42.0,0,0,0,40대,10721.0,수지구,신봉동,0,3,1,매장C,0.333333,0.0,0.0,0.0,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.333333,0.666667,1.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4365,998576256,1,0,39.0,0,0,0,30대,10434.0,수지구,풍덕천동,0,3,0,매장C,0.000000,0.0,0.0,0.0,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.000000,0.666667,0.333333,1.000000,2
4366,99912192,1,0,64.0,0,0,0,60대,94764.0,수지구,성복동,1,11,1,매장C,0.133333,0.0,0.0,0.0,0.533333,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.200000,0.000000,0.454545,0.545455,1.363636,7
4367,999513536,1,0,38.0,0,0,0,30대,55709.0,수지구,풍덕천동,0,4,0,매장C,0.000000,0.0,0.0,0.0,0.111111,0.000000,0.000000,0.000000,0.000000,0.222222,0.000000,0.0,0.0,0.666667,0.000000,1.000000,0.000000,2.250000,2
4368,999569216,1,0,42.0,0,0,0,40대,28238.0,기흥구,보정동,0,3,1,매장C,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.0,0.0,0.500000,0.000000,0.666667,0.333333,1.333333,8


In [45]:
dist_D = pd.read_csv('D매장_거리.csv')
dist_D['주소'] = dist_D.위치
dist_D.drop('위치', axis = 1, inplace = True)
dist_D = dist_D.merge(mem_add, on = '주소')
mD = memberD.merge(dist_D.loc[:,['거리','회원번호']], on = '회원번호' , how = 'left')
mD

,회원번호,조합원상태,성별,연령,배송서비스신청여부,모바일알람여부,Gold_member,연령대,구매금액,구,동,연령60이상,방문횟수,연령대_4560대,구매매장,간식,건강,과실,기타,반찬,생활용품,서류,수산,양념/가루,음료,잡곡,주곡,주류,채소,축산물,주말,평일,1회당평균구매물품수,거리
0,1000330176,1,0,64.0,0,0,0,60대,317217.0,수지구,신봉동,1,20,1,매장D,0.111111,0.0,0.0000,0.0,0.185185,0.000000,0.037037,0.000000,0.185185,0.000000,0.000000,0.148148,0.0,0.111111,0.222222,0.200000,0.800000,1.350000,2
1,1001592256,1,0,40.0,0,0,0,40대,38288.0,수지구,성복동,0,1,1,매장D,0.000000,1.0,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,1.000000,7
2,1001712896,1,0,49.0,0,0,0,40대,37523.0,수지구,신봉동,0,3,1,매장D,0.000000,0.0,0.0000,0.0,0.250000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.500000,0.666667,0.333333,1.333333,2
3,1001750016,1,0,44.0,0,0,0,40대,3637.0,분당구,백현동,0,1,1,매장D,1.000000,0.0,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,1.000000,17
4,1003019520,1,0,72.0,0,0,0,70대이상,80022.0,기타,기타,1,5,0,매장D,0.222222,0.0,0.0000,0.0,0.111111,0.222222,0.000000,0.111111,0.111111,0.000000,0.111111,0.111111,0.0,0.000000,0.000000,0.400000,0.600000,1.800000,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2034,997909952,1,0,47.0,0,0,0,40대,139465.0,분당구,중동,0,14,1,매장D,0.187500,0.0,0.0625,0.0,0.062500,0.000000,0.000000,0.187500,0.062500,0.062500,0.000000,0.000000,0.0,0.187500,0.187500,0.285714,0.714286,1.142857,20
2035,998583680,1,0,43.0,0,0,0,40대,45276.0,서울 강남구,기타,0,2,1,매장D,0.500000,0.0,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,1.000000,2.000000,29
2036,998990144,1,0,40.0,0,0,0,40대,11486.0,수지구,신봉동,0,1,1,매장D,1.000000,0.0,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,2.000000,2
2037,999580352,1,0,63.0,0,0,0,60대,50925.0,수지구,신봉동,1,5,1,매장D,0.000000,0.0,0.0000,0.0,0.222222,0.000000,0.000000,0.000000,0.111111,0.666667,0.000000,0.000000,0.0,0.000000,0.000000,0.600000,0.400000,1.800000,2


In [46]:
mA.to_csv('memberA.csv', index = False)
mB.to_csv('memberB.csv', index = False)
mC.to_csv('memberC.csv', index = False)
mD.to_csv('memberD.csv', index = False)